In [ ]:
from neural_map import NeuralMap, _plot
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.concat([pd.read_csv("datasets/table_219.csv"), pd.read_csv("datasets/table_220.csv"), pd.read_csv("datasets/table_221.csv")]).drop(['Sex', 'Delta 15 N (o/oo)', 'Delta 13 C (o/oo)', 'Comments'], axis=1).dropna().reset_index(drop=True)
l_columns = ['studyName', 'Sample Number', 'Species', 'Region', 'Island', 'Stage', 'Individual ID', 'Clutch Completion', 'Date Egg']
col = 'Species'

In [ ]:
scaler = MinMaxScaler()
data = scaler.fit_transform(df.drop(l_columns, axis=1).values)

data.shape

In [ ]:
l_columns = ['studyName', 'Sample Number', 'Species', 'Region', 'Island', 'Stage', 'Individual ID', 'Clutch Completion', 'Date Egg']
df.drop(columns=l_columns)

In [ ]:
som = NeuralMap(variables=data.shape[1], columns=6, rows=6, metric='correlation')

In [ ]:
som.train(data=data, n_epochs=20, verbose=False, seed=None)

In [ ]:
som.plot_analysis(data, attached_values=df[col], size=8)

In [ ]:
clusters = 4

labels, centers = som.k_means(clusters)
_plot.tiles(som.positions, som.hexagonal, labels, norm=False, title='K-means clustering   K = ' + str(clusters), labels=list(range(clusters)), size=8)

labels, centers = som.k_medoids(clusters)
_plot.tiles(som.positions, som.hexagonal, labels, norm=False, title='K-medoids clustering   K = ' + str(clusters), labels=list(range(clusters)), size=8)

In [ ]:
som.plot_weights(scaler=scaler, headers=df.drop(columns=l_columns).columns, size=8, show_all=False)

In [ ]:
som.plot_unified_distance_matrix(size=8, detailed=False)

In [ ]:
som.plot_analysis(som.weights.reshape((-1, som.variables)), title=df.drop(columns=l_columns).columns[0], size=8, attached_values=som.weights.reshape((-1, som.variables))[:, 0], aggregation_function=np.mean)

In [ ]:
som.plot_cluster_weights_vectors(1, xticks_labels=df.drop(columns=l_columns).columns, display_median=False, display_mean=True, display_lines=True ,size=8)

In [ ]:
som.plot_weights_vector((0, 0), xticks_labels=df.drop(columns=l_columns).columns, line=True, scatter=True, size=8)